In [294]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time

options = webdriver.ChromeOptions()

# set the window size
options.add_argument('window-size=1200x600')

# initialize the driver
driver = webdriver.Chrome(chrome_options=options)

# https://www.linkedin.com/uas/login
driver.get('https://www.linkedin.com/uas/login')

#find the username and password forms on the page
username = driver.find_element_by_id("session_key-login")
password = driver.find_element_by_id("session_password-login")

#fill in the details
username.send_keys("lloyd@growthgenius.com")
password.send_keys("GrowthGenius1")
# username.send_keys("g2lei@edu.uwaterloo.ca")
# password.send_keys("Accelerator")

#submit
driver.find_element_by_name("signin").click()

#note: unable to automatically fill in captcha's, will have to do that manually

In [295]:
#import pandas for data analysis

import pandas as pd

#import csv
df = pd.read_csv("linkedin_outbound_lloyd.csv")

In [296]:
#filter out the NaNs
print(len(df))
df = df[df["Person Linkedin Url"].notnull()]
length_df = len(df)
print(length_df)

639
324


In [297]:
#scroll to the end of page
def scroll_to_end_of_page():
    scrolled_amount = 0
    height = driver.execute_script("return document.body.scrollHeight")
    while scrolled_amount < height:
        scrolled_amount+=100
        driver.execute_script("window.scrollTo(0, %s);" % scrolled_amount)
        time.sleep(0.2)
        height = driver.execute_script("return document.body.scrollHeight")

In [ ]:
#grab name, connectiontype, headline or location
def get_name():
    name=driver.find_element_by_xpath('//h1[1]')
    return name.text

def get_connection():
    connection_type=driver.find_element_by_xpath('//span[@class="dist-value"]')
    return connection_type.text

def get_headline():
    headline=driver.find_element_by_xpath('//h2[1]')
    return headline.text

def get_location():
    location=driver.find_element_by_xpath('//h3[1]')
    return location.text

In [ ]:
#grab description

def get_description():
    full_description = ""
    
    #check for a description
    try:
        description_box = driver.find_element_by_xpath('//div/p[@class="pv-top-card-section__summary-text mt4 ember-view"]/span')
        description_exist = True
    except NoSuchElementException:
        description_exist = False

    #     If there is a show more button for description click it and then grab, otherwise just grab the description
    if(description_exist):
        try:
            driver.find_element_by_css_selector('button.pv-top-card-section__summary-toggle-button.pv-profile-section__card-action-bar.artdeco-container-card-action-bar.mt4').click()
            full_description = driver.find_element_by_xpath('//p[@class="pv-top-card-section__summary-text text-align-left mt4 ember-view"]')
            full_description = full_description.text
        except NoSuchElementException:
            all_descriptions = driver.find_elements_by_xpath('//div/p/span')
            for description in all_descriptions:
                full_description = full_description + description.text

    return full_description

In [ ]:
 #mutual Connections
    
def get_num_mutuals():
    mutual_exists = True
    try:
        total_mutual_connections=driver.find_element_by_xpath('//h3[@class="pv-highlight-entity__primary-text Sans-17px-black-85%-semibold"]')
    except NoSuchElementException:
        mutual_exists = False
        total_mutual_connections = 0
    if mutual_exists == True:
        total_mutual_connections = total_mutual_connections.text.split(" ")[0]
    return total_mutual_connections

In [298]:
#present jobs

def get_present_job():
    attempts = 0;
    while(attempts<2):
        try:
            present_title = driver.find_element_by_xpath("//section[@id='experience-section']//li[1]//h3[1]")
            present_job['Title'] = present_title.text
            
            present_company = driver.find_element_by_xpath("//section[@id='experience-section']//li[1]//h4[1]/span[2]")
            present_job['Company'] = present_company.text
            
            present_dates = driver.find_element_by_xpath("//section[@id='experience-section']//li[1]//h4[2]/span[2]")
            present_job['Dates'] = present_dates.text

            present_location = driver.find_element_by_xpath("//section[@id='experience-section']//li[1]//h4[4]/span[2]")
            present_job['Location'] = present_location.text

            present_description = driver.find_element_by_xpath("//section[@id='experience-section']//li[1]//div/p")
            present_job['Description'] = present_description.text
            break
        except (StaleElementReferenceException, NoSuchElementException) as e:
            if e == StaleElementReferenceException:
                attempts += 1
            else:
                attempts = 4

In [299]:
#past jobs

def get_past_job(take_first):
    attempts = 0;
    while(attempts<2):
        try:
            past_title = driver.find_element_by_xpath("//section[@id='experience-section']//li[%s]//h3[1]" % take_first)
            past_job['Title'] = past_title.text
            
            past_company = driver.find_element_by_xpath("//section[@id='experience-section']//li[%s]//h4[1]/span[2]" % take_first)
            past_job['Company'] = past_company.text
            
            past_dates = driver.find_element_by_xpath("//section[@id='experience-section']//li[%s]//h4[2]/span[2]" % take_first)
            past_job['Dates'] = past_dates.text

            past_location = driver.find_element_by_xpath("//section[@id='experience-section']//li[%s]//h4[4]/span[2]" % take_first)
            past_job['Location'] = past_location.text

            past_description = driver.find_element_by_xpath("//section[@id='experience-section']//li[%s]//div/p" % take_first)
            past_job['Description'] = past_description.text
            break
        except (StaleElementReferenceException, NoSuchElementException) as e:
            if e == StaleElementReferenceException:
                attempts += 1
            else:
                attempts = 4

In [300]:
#job helper

def job_helper():
    
    take_first = 2
    current_date=""
    
    #check for experience section
    try:
        all_exper = driver.find_elements_by_xpath("//section[@id='experience-section']/ul/li")
        experience_exists = True
    except NoSuchElementException:
        experience_exists = False
        
    #continue if there exists an experience section    
    if all_exper:
        number_experiences = len(all_exper)
        if number_experiences > 0:
            present_dates = driver.find_element_by_xpath("//section[@id='experience-section']//li[1]//h4[2]/span[2]")
            date_list = present_dates.text.split(" ")
            current_date = date_list[-1:]
            if current_date == ['Present']:
                get_present_job()
            else:
                #if not go straight to past
                take_first = 1
                number_experiences = 2
                
        if number_experiences > 1:
            get_past_job(take_first)


In [301]:
#figure out how many times to iterate
number_connection_requests = 1
max_number_conn_requests = 2

if max_number_conn_requests > length_df:
    max_number_conn_requests = length_df

In [302]:
#scrap and save
for x in df["Person Linkedin Url"].tolist():
    if number_connection_requests > max_number_conn_requests:
        break
    #head to url, continue if it works
    driver.get(x)
    if driver.current_url != "https://www.linkedin.com/in/unavailable/":
        
        #scrap information

        #name, connection type, headline, location
        name = get_name()
        print(name)
        connection_type = get_connection()
        print(connection_type)
        headline = get_headline()
        print(headline)
        location = get_location()
        print(location)

        #description
        one_description = ""

        one_description = get_description()
        print(one_description)

        #mutual connections
        total_mutual_connections = get_num_mutuals()
        print(total_mutual_connections)
                    
        #scroll
        scroll_to_end_of_page()

        #present and past jobs
        present_job = {
            'Title' : "",
            'Company' : "",
            'Dates' : "",
            'Location' : "",
            'Description' : ""
            }

        past_job = {
                'Title' : "",
                'Company' : "",
                'Dates' : "",
                'Location' : "",
                'Description' : ""
            }
        job_helper()
        
        #save to dictionary
        datastore={
            'Name' : name,
            'Connection' : connection_type,
            'Headline' : headline,
            'Location' : location,
            'Description' : one_description,
            'Mutuals' : total_mutual_connections,
            'Present Job' : present_job,
            'Past Job' : past_job
        }
    
    #complete move to next
    number_connection_requests += 1


#LIMIT TO 30 connection requests

DESCRIPTion PassKey is the fastest growing and #1 Security app on iOS - Join over 10 million people using PassKey to upgrade security & replace typing passwords on every device with your fingerprint or Face ID.
DESCRIPTion Tayfun provides financial, marketing, business development and strategic expertise to founders and senior management of tech companies in valuation, pricing, sales development, M&A, FP&A, SaaS projects.

Selected Toptal Projects:

- Sales Development Team Analysis - Australia:
Project involved creation of a hypothetical sales team that includes inside Sales reps and account managers and formation of a flexible model that considers lead management, sales development and their financial impact on P&L.

- Mobile eCommerce app - Gulf Region:
Analyzed pricing and suggested freemium and gradual pricing options for a regional mobile SaaS eCommerce player to increase conversion and retention based on Microsoft BI output. Prepared a flexible forecasting model to compare new a

In [ ]:
#past and present jobs

# def get_jobs(number_experiences):
#     #check for experience section

#     take_first = 2
#     current_date=""
    
#     #if there exists experiences go for present jobs then past
#     if number_experiences > 0:
#         attempts = 0;
#         while(attempts<2):
#             #check if first experience is the current experience
#             present_dates = driver.find_element_by_xpath("//section[@id='experience-section']//li[1]//h4[2]/span[2]")
#             date_list = present_dates.text.split(" ")
#             current_date = date_list[-1:]
#             if current_date == ['Present']:
#                 found_present = True
#                 try:
#                     present_title = driver.find_element_by_xpath("//section[@id='experience-section']//li[1]//h3[1]")
#                     present_title = present_title.text
#                     print(present_title)
#                     present_company = driver.find_element_by_xpath("//section[@id='experience-section']//li[1]//h4[1]/span[2]")
#                     present_company = present_company.text
#                     print(present_company)
#                     present_dates = driver.find_element_by_xpath("//section[@id='experience-section']//li[1]//h4[2]/span[2]")
#                     present_dates = present_dates.text
#                     print(present_dates)
#                     present_location = driver.find_element_by_xpath("//section[@id='experience-section']//li[1]//h4[4]/span[2]")
#                     present_location = present_location.text
#                     print(present_location)
#                     present_description = driver.find_element_by_xpath("//section[@id='experience-section']//li[1]/div/p")
#                     present_description = present_description.text
#                     print(present_description)
#                 except (StaleElementReferenceException, NoSuchElementException):
#                     attempts += 1
#                 break
#             else:
#                 #if not go straight to past
#                 take_first = 1
#                 number_experiences = 2
#                 present_dates = ""
#                 break
#     else:
#         initiate_present_jobs()
#         if number_experiences > 1:
#             attempts = 0;
#             while(attempts<2):
#                 try:
#                     past_title = driver.find_element_by_xpath("//section[@id='experience-section']//li[%s]//h3[1]" % take_first)
#                     past_title = past_title.text
#                     print(past_title)
#                     past_company = driver.find_element_by_xpath("//section[@id='experience-section']//li[%s]//h4[1]/span[2]" % take_first)
#                     past_company = past_company.text
#                     print(past_company)
#                     past_dates = driver.find_element_by_xpath("//section[@id='experience-section']//li[%s]//h4[2]/span[2]" % take_first)
#                     past_dates = past_dates.text
#                     print(past_dates)
#                     past_location = driver.find_element_by_xpath("//section[@id='experience-section']//li[%s]//h4[4]/span[2]" % take_first)
#                     past_location = past_location.text
#                     print(past_location)
#                     past_description = driver.find_element_by_xpath("//section[@id='experience-section']//li[%s]/div/p" % take_first)
#                     past_description = past_description.text
#                     print(past_description)
#                     break
#                 except (StaleElementReferenceException, NoSuchElementException) as e:
#                     if e == StaleElementReferenceException:
#                         attempts += 1
#                     else:
#                         attempts = 4
#         else:
#             initiate_past_jobs()